In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.base import ClassifierMixin, OutlierMixin, clone
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [2]:
data_path = Path("data/")

## MNIST

In [3]:
df = pd.read_parquet(data_path / "mnist" / "mnist.parquet")

In [4]:
df.columns

Index(['class', 'split', 'image', 'clip', 'clip_norm'], dtype='object')

In [5]:
train_mask = df["split"] == "train"
test_mask = df["split"] == "test"
X_train = np.array(df.loc[train_mask, "clip_norm"].tolist())
X_test = np.array(df.loc[test_mask, "clip_norm"].tolist())
y_train = df.loc[train_mask, "class"].to_numpy()
y_true = df.loc[test_mask, "class"].to_numpy()

In [6]:
classifier = KNeighborsClassifier(n_neighbors=15)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_true, y_pred)

0.9641

In [7]:
scorer = LocalOutlierFactor(n_neighbors=15, novelty=True)
scorer.fit(X_train)
decision_pred = scorer.predict(X_test)
accuracy_score(np.ones(decision_pred.size), decision_pred)

0.9955

In [8]:
def train_test_with_class_holdout(X_train: np.ndarray[float],
                                  y_train: np.ndarray[int],
                                  X_test: np.ndarray[float],
                                  y_true: np.ndarray[int],
                                  classifier: ClassifierMixin,
                                  scorer: OutlierMixin) -> pd.DataFrame:
    """For each class trains a model holding out that class.

    Args:
      X_train: Array of shape (n_samples, n_features).
      y_train: Array of shape (n_samples,).

    Returns:
      A dataframe with the held-out class, trained classifier and scorer.
    """
    models = []
    labels = np.sort(np.unique(y_train))
    for label in tqdm(labels):
        label_mask = df.loc[train_mask, "class"] != label
        holdout_classifier = clone(classifier)
        holdout_classifier.fit(X_train[label_mask, :], y_train[label_mask])
        holdout_scorer = clone(scorer)
        holdout_scorer.fit(X_train[label_mask, :])

        decision_true = np.ones(y_true.shape, dtype=int)
        decision_true[y_true == label] = -1

        models.append({
            "class": label,
            "scorer": holdout_scorer,
            "classifier": holdout_classifier,
            "scores": holdout_scorer.score_samples(X_test),
            "decision_pred": holdout_scorer.predict(X_test),
            "decision_true": decision_true,
            "y_pred": holdout_classifier.predict(X_test),
        })
    models = pd.DataFrame(models)
    return models

In [9]:
classifier = KNeighborsClassifier(n_neighbors=15)
scorer = LocalOutlierFactor(n_neighbors=15, novelty=True)
models = train_test_with_class_holdout(X_train, y_train, X_test, y_true,
                                       classifier, scorer)

100%|██████████| 10/10 [00:56<00:00,  5.65s/it]


In [10]:
[accuracy_score(y_true, y_pred) for y_pred in models["y_pred"]]

[0.8689, 0.8532, 0.8685, 0.871, 0.871, 0.8823, 0.8744, 0.869, 0.8727, 0.8716]

In [11]:
[
    accuracy_score(true, pred)
    for true, pred in models[["decision_true", "decision_pred"]].values.tolist()
]

[0.912, 0.8944, 0.9003, 0.8956, 0.9109, 0.9125, 0.9065, 0.9148, 0.9033, 0.8993]

In [12]:
models

,class,scorer,classifier,scores,decision_pred,decision_true,y_pred
0,0,"LocalOutlierFactor(n_neighbors=15, novelty=True)",KNeighborsClassifier(n_neighbors=15),"[-1.0214984956166597, -1.1256783605807401, -1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1,...","[1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, ...","[7, 2, 1, 8, 4, 1, 4, 9, 5, 9, 9, 6, 9, 8, 1, ..."
1,1,"LocalOutlierFactor(n_neighbors=15, novelty=True)",KNeighborsClassifier(n_neighbors=15),"[-1.0214984956166597, -1.1256783605807401, -1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -...","[7, 2, 7, 0, 4, 7, 4, 9, 5, 9, 0, 6, 9, 0, 7, ..."
2,2,"LocalOutlierFactor(n_neighbors=15, novelty=True)",KNeighborsClassifier(n_neighbors=15),"[-1.0214984956166597, -1.4558664295340475, -1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[7, 7, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, ..."
3,3,"LocalOutlierFactor(n_neighbors=15, novelty=True)",KNeighborsClassifier(n_neighbors=15),"[-1.0214984956166597, -1.1256783605807401, -1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, ..."
4,4,"LocalOutlierFactor(n_neighbors=15, novelty=True)",KNeighborsClassifier(n_neighbors=15),"[-1.0214984956166597, -1.1256783605807401, -1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1...","[7, 2, 1, 0, 9, 1, 2, 9, 5, 9, 0, 6, 9, 0, 1, ..."
5,5,"LocalOutlierFactor(n_neighbors=15, novelty=True)",KNeighborsClassifier(n_neighbors=15),"[-1.0214984956166597, -1.1256783605807401, -1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1,...","[7, 2, 1, 0, 4, 1, 4, 9, 3, 9, 0, 6, 9, 0, 1, ..."
6,6,"LocalOutlierFactor(n_neighbors=15, novelty=True)",KNeighborsClassifier(n_neighbors=15),"[-1.0214984956166597, -1.1256783605807401, -1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1,...","[7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 0, 9, 0, 1, ..."
7,7,"LocalOutlierFactor(n_neighbors=15, novelty=True)",KNeighborsClassifier(n_neighbors=15),"[-1.5210974183668, -1.1256783605807401, -1.009...","[-1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[-1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[9, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, ..."
8,8,"LocalOutlierFactor(n_neighbors=15, novelty=True)",KNeighborsClassifier(n_neighbors=15),"[-1.0214984956166597, -1.1256783605807401, -1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, ..."
9,9,"LocalOutlierFactor(n_neighbors=15, novelty=True)",KNeighborsClassifier(n_neighbors=15),"[-1.0214984956166597, -1.1256783605807401, -1....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, ...","[7, 2, 1, 0, 4, 1, 4, 8, 5, 7, 0, 6, 4, 0, 1, ..."


## Pets

In [13]:
df = pd.read_parquet(data_path / "pets" / "pets.parquet")

In [14]:
df.columns

Index(['path', 'image', 'clip', 'clip_norm', 'class', 'species', 'image_n',
       'split'],
      dtype='object')

In [15]:
label_encoder = LabelEncoder()
label_encoder.fit(df["species"])
y_true = label_encoder.transform(df["species"])